# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
%matplotlib inline
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv("output_data/cities.csv")
vacation_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.12,-134.97,70.88,78,8,13.49,PF,2020-08-17 17:08:02
1,boulder city,35.98,-114.83,100.99,22,1,9.17,US,2020-08-17 17:12:40
2,ushtobe,45.25,77.98,77.68,32,68,10.31,KZ,2020-08-17 17:12:40
3,santo antonio do ica,-3.10,-67.94,98.76,37,15,2.26,BR,2020-08-17 17:12:40
4,mujiayingzi,42.12,118.78,69.76,84,44,1.86,CN,2020-08-17 17:12:40


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
vacation_df

locations = vacation_df[["Lat", "Lng"]]
humidity = vacation_df["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=200,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# **Note:** Feel free to adjust to your specifications but be sure to limit the number of rows returned 
#                 by your API requests to a reasonable number.

# * Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

# * Plot the hotels on top of the humidity heatmap with each pin containing t

In [6]:
criteria_df = vacation_df.loc[(vacation_df["Max Temp"] < 80) & (vacation_df["Max Temp"] > 70) & (vacation_df["Wind Speed"] < 10)\
                        & (vacation_df["Cloudiness"] == 0)]

In [7]:
criteria_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,xihe,31.69,113.47,79.36,84,0,5.84,CN,2020-08-17 17:12:43
129,talas,38.69,35.55,73.40,35,0,3.36,TR,2020-08-17 17:12:55
387,chirongui,-12.93,45.15,75.20,83,0,3.36,YT,2020-08-17 17:13:28
407,katherine,-14.47,132.27,78.80,29,0,6.93,AU,2020-08-17 17:09:11
415,port keats,-14.25,129.55,73.90,91,0,6.71,AU,2020-08-17 17:13:32
472,binga,-17.62,27.34,72.41,30,0,3.53,ZW,2020-08-17 17:13:40
486,mersing,2.43,103.84,75.20,94,0,5.82,MY,2020-08-17 17:13:42
488,le mars,42.79,-96.17,78.01,50,0,1.01,US,2020-08-17 17:13:42


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = criteria_df[["City", "Country", "Lat", "Lng"]].copy()

In [9]:
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
27,xihe,CN,31.69,113.47,
129,talas,TR,38.69,35.55,
387,chirongui,YT,-12.93,45.15,
407,katherine,AU,-14.47,132.27,
415,port keats,AU,-14.25,129.55,
472,binga,ZW,-17.62,27.34,
486,mersing,MY,2.43,103.84,
488,le mars,US,42.79,-96.17,


In [10]:
params = {
    "radius": 5000,
    "types": "Lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
27,xihe,CN,31.69,113.47,
129,talas,TR,38.69,35.55,Radisson Blu Hotel Kayseri
387,chirongui,YT,-12.93,45.15,Les Baobabs
407,katherine,AU,-14.47,132.27,Beagle Motor Inn
415,port keats,AU,-14.25,129.55,TDC Village (Hundred Man Camp)
472,binga,ZW,-17.62,27.34,Masumu River Lodge
486,mersing,MY,2.43,103.84,Riverside Hotel
488,le mars,US,42.79,-96.17,Holiday Inn Express & Suites Le Mars


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))